In [87]:
import importlib.util
import subprocess
import sys

def install_package(package_name):
    subprocess.check_call([sys.executable, "-m", "pip", "install", package_name])

def check_package(package_name):
    spec = importlib.util.find_spec(package_name)
    if spec is None:
        print(f"{package_name} is not installed. Installing...")
        install_package(package_name)
        print(f"...ready.")
    else:
        print(f"{package_name} is already installed.")

check_package("pyfinance")

pyfinance is already installed.


In [88]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import norm
from pyfinance.options import BSM
import pandas as pd

import numpy as np

def pricediff(sigma, S, r, T, E, Ubs):
    d1 = (np.log(S/E) + (r + sigma ** 2 / 2) * T) / (sigma * np.sqrt(T))
    d2 = (np.log(S/E) + (r - sigma ** 2 / 2) * T) / (sigma * np.sqrt(T))
    Nd_plus= norm.cdf(d1)
    Nd_minus = norm.cdf(d2)
    C = S*Nd_plus - E * np.exp(-r*T)*Nd_minus
    a = Ubs - C
    return a

In [89]:
def norehedge(S1, S2, C1, C2, delta):
    diffRE = delta * (S2 - S1)
    diffOG = C2 - C1
    A0 = diffOG - diffRE
    return A0

def rehedge(S1, S2, C1, C2, delta, r, T, E):
    diffRE = delta * (S2 - S1)
    diffOG = C2 - C1
    A0 = diffOG - diffRE
    #computing new volatility
    a =0.00000001
    b = 1
    for i in range (30):
        m1 = 0.5*(a + b)
        h = pricediff(m1, S2, r, T, K, C2) 
        if(h<0):
            b = m1
        else:
            a = m1
        i = i+1
    d1 = (np.log(S2/E) + (r + m1 ** 2 / 2) * T) / (m1 * np.sqrt(T)) #from bse
    delta_new = norm.cdf(d1)   
    return delta_new, A0

In [92]:
df = pd.read_csv("IBM_2023-08-18_processed_45.csv")

S = df["S"].values #underlying
T = df["TTM"].values #time from maturity
C = df["C"].values #call price
r = df["r"].values #risk ratio
K = df["K"].values[0] #risk ratio

a =0.00000001
b = 1

A = np.zeros(len(T), dtype=float)

#compute first volatility
for i in range (30):
    m1 = 0.5*(a + b)
    h = pricediff(m1, S[0], r[0], T[0], K, C[0])
    if(h<0):
        b = m1
    else:
        a = m1

#compute first delta
d1 = (np.log(S[0]/K) + (r[0] + m1 ** 2 / 2) * T[0]) / (m1 * np.sqrt(T[0])) #from bse
delta0 = norm.cdf(d1)

print(delta0)
print( S[0], r[0], T[0], K)
print(m1)

#portfolio OP
OP0 = C[0]

#portfolio RE
RE0 = delta0 * S[0]

for i in range(1, len(T)-1):
    if i%2 != 0: #if odd day dont rehedge
        A[i] = norehedge(S[i-1], S[i], C[i-1], C[i], delta0)
    else: 
        delta0, A[i] = rehedge(S[i-1], S[i], C[i-1], C[i], delta0, r[i], T[i], K)

0.7484446568959056
134.24 0.0519717641748631 0.1205479452054794 130.0
0.1727338860581651


In [93]:
print(A)

[ 0.          0.43676489 -0.15012443 -0.37909094  0.38634141 -0.08118239
 -0.13245189 -0.11836993  0.33352065 -0.14285242  0.09290867 -0.8898308
  0.77748095 -0.45828918 -0.04191409  0.23       -0.61        0.03
  0.08       -0.46        0.61        0.47       -0.49       -0.4
  0.33        0.97       -0.53        0.27675096 -0.64275203  0.81848301
  0.03067507  0.        ]


In [94]:
mse = np.sum(A**2) / (len(A) - 2)
print(mse)

0.21403429438623664
